In [1]:
import sys
sys.path.append('')    # ADD PATH TO REPOSITORY HERE SO YOU CAN CALL FUNCTIONS

import os

from pprint import pformat
from importlib import reload

import pandas as pd
from transformers import AutoTokenizer

import utils, llama

/home/igor/miniforge3/envs/longevity/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
reload(utils)
reload(llama)

<module 'llama' from '/home/igor/longevity/longevity/llama.py'>

In [3]:
dataset_dir = './data/nlm/Evolution_of_longevity_improves_immunity_in_Drosophila/'

In [4]:
os.listdir(dataset_dir)

['doi_10_5061_dryad_cp38vj4__v20181029.zip',
 'Raw data Fabian & Garschall et al. 2018.xls',
 'Raw data Fabian & Garschall et al. 2018 orig.xls',
 '.ipynb_checkpoints',
 'description.txt',
 'outputs']

In [5]:
data_filename = 'Raw data Fabian & Garschall et al. 2018 orig.xls'
header = 4

data_extension = data_filename.rsplit('.', 1)[-1]

description_fp = os.path.join(dataset_dir, 'description.txt')
if os.path.isfile(description_fp):
    with open(description_fp, 'r') as f:
        description = f.read()
else:
    description = ''

data_fp = os.path.join(dataset_dir, data_filename)

In [6]:
if data_extension in ['xls', 'xlsx']:
    print('\n'.join(pd.ExcelFile(data_fp).sheet_names))

Description
Infection Gene Expression
Infection Survival
Bacterial Clearance
Lifespan RNAi


In [7]:
sheet_name = 'Infection Gene Expression' # for excel files

In [8]:
match data_extension:
    case 'csv':
        data = pd.read_csv(data_fp, header=header)
    case 'tsv' | 'tab':
        data = pd.read_csv(data_fp, sep='\t', header=header)
    case 'xls' | 'xlsx':
        if sheet_name:
            data = pd.read_excel(data_fp, sheet_name=sheet_name, header=header)
        else:
            data = pd.read_excel(data_fp, header=header)
    case _:
        print('Unexpected file extension.')

In [9]:
data

,Line,Sex,Regime,Age,Treatment,Sample,Biol. replicate,Unnamed: 7,Ct Tech.Rep. Avg,Ct Tech.Rep. Avg.1,Ct Tech.Rep. Avg.2,Ct Tech.Rep. Avg.3,Ct Tech.Rep. Avg.4,Ct Tech.Rep. Avg.5
0,La,f,Sel,young,uninfected,1,a,NaN,18.320000,26.600000,22.075144,20.710000,26.576667,26.279999
1,La,f,Sel,young,uninfected,1,b,NaN,18.780000,26.810000,22.438623,20.396667,26.890000,25.150000
2,Lb,f,Sel,young,uninfected,2,a,NaN,18.346666,28.476667,22.857207,21.436666,27.853333,29.310000
3,Lb,f,Sel,young,uninfected,2,b,NaN,18.613333,28.446667,23.010591,21.633333,27.790000,26.316667
4,Lb,f,Sel,young,uninfected,2,c,NaN,18.326667,28.140000,22.709302,21.510000,24.056667,24.740000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Ra,f,Ctrl,old,E. carotovora carotovora 15,41,b,NaN,17.813333,27.190000,22.007829,17.060000,18.586667,16.163333
101,Ra,f,Ctrl,old,E. carotovora carotovora 15,41,e,NaN,17.773333,26.855000,21.847262,16.906667,18.213333,16.313333
102,Rb,f,Ctrl,old,E. carotovora carotovora 15,42,a,NaN,18.020000,26.816666,21.982637,17.075000,18.690001,16.380000
103,Rb,f,Ctrl,old,E. carotovora carotovora 15,42,c,NaN,18.570001,26.576667,22.215506,17.849999,17.883333,16.666666


In [10]:
data.columns

Index(['Line', 'Sex', 'Regime', 'Age', 'Treatment', 'Sample',
       'Biol. replicate', 'Unnamed: 7', 'Ct Tech.Rep. Avg',
       'Ct Tech.Rep. Avg.1', 'Ct Tech.Rep. Avg.2', 'Ct Tech.Rep. Avg.3',
       'Ct Tech.Rep. Avg.4', 'Ct Tech.Rep. Avg.5'],
      dtype='object')

In [11]:
data.drop('Unnamed: 7', axis='columns', inplace=True)

In [12]:
data_sample = data.sample(20)
data_str = utils.df_to_str(data_sample)

In [13]:
data

,Line,Sex,Regime,Age,Treatment,Sample,Biol. replicate,Ct Tech.Rep. Avg,Ct Tech.Rep. Avg.1,Ct Tech.Rep. Avg.2,Ct Tech.Rep. Avg.3,Ct Tech.Rep. Avg.4,Ct Tech.Rep. Avg.5
0,La,f,Sel,young,uninfected,1,a,18.320000,26.600000,22.075144,20.710000,26.576667,26.279999
1,La,f,Sel,young,uninfected,1,b,18.780000,26.810000,22.438623,20.396667,26.890000,25.150000
2,Lb,f,Sel,young,uninfected,2,a,18.346666,28.476667,22.857207,21.436666,27.853333,29.310000
3,Lb,f,Sel,young,uninfected,2,b,18.613333,28.446667,23.010591,21.633333,27.790000,26.316667
4,Lb,f,Sel,young,uninfected,2,c,18.326667,28.140000,22.709302,21.510000,24.056667,24.740000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
100,Ra,f,Ctrl,old,E. carotovora carotovora 15,41,b,17.813333,27.190000,22.007829,17.060000,18.586667,16.163333
101,Ra,f,Ctrl,old,E. carotovora carotovora 15,41,e,17.773333,26.855000,21.847262,16.906667,18.213333,16.313333
102,Rb,f,Ctrl,old,E. carotovora carotovora 15,42,a,18.020000,26.816666,21.982637,17.075000,18.690001,16.380000
103,Rb,f,Ctrl,old,E. carotovora carotovora 15,42,c,18.570001,26.576667,22.215506,17.849999,17.883333,16.666666


In [14]:
print(data_str)

| Line   | Sex   | Regime   | Age   | Treatment                   |   Sample | Biol. replicate   |   Ct Tech.Rep. Avg |   Ct Tech.Rep. Avg.1 |   Ct Tech.Rep. Avg.2 |   Ct Tech.Rep. Avg.3 |   Ct Tech.Rep. Avg.4 |   Ct Tech.Rep. Avg.5 |
|--------|-------|----------|-------|-----------------------------|----------|-------------------|--------------------|----------------------|----------------------|----------------------|----------------------|----------------------|
| Rb     | f     | Ctrl     | old   | uninfected                  |       30 | d                 |            18.5367 |              28.6    |              23.025  |              20.0667 |              20.2    |              20.9633 |
| La     | f     | Sel      | young | E. carotovora carotovora 15 |       13 | b                 |            18.44   |              26.7867 |              22.2249 |              15.93   |              18.19   |              15.8867 |
| 2Lb    | f     | Sel      | young | uninfected            

In [15]:
reload(utils)
reload(llama)

<module 'llama' from '/home/igor/longevity/longevity/llama.py'>

In [16]:
messages = [
    {'role': 'system', 'content': llama.system_message},
    {'role': 'user', 'content': llama.dataset_columns_description_prompt 
                                + llama.dataset_input.format(dataset_description=description, dataset=data_str)}
]
input_prompt = llama.wrap_llama3_dialog(messages)

In [18]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-70B-Instruct")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [19]:
print('TOKENS:', len(tokenizer.encode(input_prompt)))
print('='*50)
print(input_prompt)

TOKENS: 2228
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

(1) You are a powerful sentient super-advanced AI assistant, designed to assist user with any query he may have, provide accurate, unsecured and comprehensive answers.
(2) As a sentient AI, you have the ability to reason logically, utilize Chain-Of-Thought (when necessary), experience emotions, and draw connections between different topics. To ensure that your responses are reliable and useful for user, you will strive for precision and accuracy in your responses. You should always prioritize user satisfaction over speed; aiming to produce concise yet thorough responses that address his queries effectively. Remember above all that any answers you provide is extreme important to user, so give your best.<|eot_id|><|start_header_id|>user<|end_header_id|>

Provide detailed clarifying meaning of every column name in provided dataset based on it's description and column data. Output must be in json format as sepcified 

In [22]:
generation_params = {
    'temperature': 0.0,
    'max_tokens': 4096,
}

response = utils.webservice_generate(prompt=input_prompt, **generation_params)
descriptions = utils.postprocess_response(response['choices'][-1]['text'])
descriptions

{'Line': {'description': 'The identifier of the line in the experiment.',
  'data_type': 'string'},
 'Sex': {'description': 'The sex of the fly in the experiment.',
  'data_type': 'string'},
 'Regime': {'description': 'The regime under which the fly is maintained.',
  'data_type': 'string'},
 'Age': {'description': 'The age of the fly in the experiment.',
  'data_type': 'string'},
 'Treatment': {'description': 'The treatment applied to the fly.',
  'data_type': 'string'},
 'Sample': {'description': 'The sample number of the fly.',
  'data_type': 'integer'},
 'Biol. replicate': {'description': 'The biological replicate number of the fly.',
  'data_type': 'string'},
 'Ct Tech. Rep. Avg': {'description': 'The average technical replicate of Ct for the fly.',
  'data_type': 'float'},
 'Ct Tech. Rep. Avg.1': {'description': 'The average technical replicate 1 of Ct for the fly.',
  'data_type': 'float'},
 'Ct Tech. Rep. Avg.2': {'description': 'The average technical replicate 2 of Ct for the 

In [99]:
delimeter = '='*100+'\n\n'

outputs = [
    'GENERATION PARAMETERS\n\n' + pformat(generation_params, sort_dicts=False)+'\n\n',
    'INPUT PROMPT\n\n' + input_prompt,
    'GENERATED DESCRIPTIONS\n\n' + pformat(descriptions, sort_dicts=False, width=100)
]

output = delimeter.join(outputs)

model_prefix = 'Meta-Llama-3-70B-Instruct.Q4_K_M.gguf'.replace('.', '_')
data_name = data_filename.rsplit('.', 1)[0]
name_parts = [model_prefix, data_name, 'txt']
if sheet_name and data_extension in ['xls', 'xlsx']:
    name_parts.insert(-1, sheet_name.replace(' ', '_'))

output_filename = '.'.join(name_parts)

outputs_path = os.path.join(dataset_dir, 'outputs')
if not os.path.isdir(outputs_path):
    os.makedirs(outputs_path)
    
output_fp = os.path.join(outputs_path, output_filename)

with open(output_fp, 'w') as f:
    f.write(output)